# 14. Conditional Maximum Loss Portfolio Optimization
This is the accompanying code to the Condtional Maximum Loss Portfolio Optimization article available at: https://antonvorobets.substack.com/p/conditional-maximum-loss-article

For a case study that illustrates how fast we can solve CML optimization problems with specialized algorithms and CML tracking error risk budgets, see https://antonvorobets.substack.com/p/conditional-maximum-loss-portfolio-optimization 

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import fortitudo.tech as ft

from scipy.optimize import linprog
from time import time
from typing import Tuple

# Download market data

In [2]:
tickers = [
    'XLB', 'XLE', 'XLF', 'XLI', 'XLK',
    'XLP', 'XLU', 'XLV', 'XLY', '^GSPC', '^VIX']
data = yf.download(tickers, start='1998-12-22', end='2026-01-24')['Close']

names_dict = {
    'XLB': 'Materials', 'XLE': 'Energy', 'XLF': 'Financial', 'XLI': 'Industrial',
    'XLK': 'Technology', 'XLP': 'Consumer Staples', 'XLU': 'Utilities',
    'XLV': 'Health Care', 'XLY': 'Consumer Discretionary', '^GSPC': 'S&P 500',
    '^VIX': 'VIX'}
data = data.rename(columns=names_dict)

print(f'The number of daily observations is {len(data)}.')

[*********************100%***********************]  11 of 11 completed

The number of daily observations is 6813.


In [3]:
data

Ticker,Materials,Energy,Financial,Industrial,Technology,Consumer Staples,Utilities,Health Care,Consumer Discretionary,S&P 500,VIX
Date,,,,,,,,,,,
1998-12-22,5.743470,5.774108,11.220836,14.472653,11.938132,14.064752,5.873081,16.858889,9.398274,1203.569946,22.780001
1998-12-23,5.803795,5.894320,11.386297,14.725199,12.223336,14.404758,5.848468,17.237747,9.438642,1228.540039,20.209999
1998-12-24,5.937361,5.863300,11.461495,14.919463,12.176774,14.379886,5.956146,17.342981,9.611615,1226.270020,21.480000
1998-12-28,5.885659,5.832277,11.311084,14.997162,12.211700,14.296954,5.925381,17.090416,9.525129,1225.489990,23.500000
1998-12-29,5.993376,5.890446,11.431417,15.210849,12.246622,14.612078,6.023831,17.469259,9.732694,1241.810059,22.180000
...,...,...,...,...,...,...,...,...,...,...,...
2026-01-16,48.680000,47.689999,54.439999,166.899994,145.619995,82.110001,43.389999,155.740005,122.300003,6940.009766,15.860000
2026-01-20,48.189999,47.599998,53.200001,163.529999,141.839996,82.360001,42.959999,155.399994,119.120003,6796.859863,20.090000
2026-01-21,49.240002,48.750000,53.459999,166.360001,143.809998,82.440002,43.020000,158.259995,121.400002,6875.620117,16.900000


# Market simulation
We use the Fully Flexible Resampling (FFR) method with VIX as state variable.

The FFR method was first introduced in Chapter 3 of the Portfolio Construction and Risk Management book: https://antonvorobets.substack.com/p/pcrm-book

For an analysis of the FFR method's properties, and more generally Time- and State-Dependent Resampling methods, see: https://ssrn.com/abstract=5117589

In [4]:
log_returns = np.diff(np.log(data.values[:, :-1]), axis=0)
print(f'Historical log returns dimension is {log_returns.shape}')

ffr = ft.FullyFlexibleResampling(log_returns)

T_tilde, I = log_returns.shape
vix_state = data['VIX'].values[1:]
probs, states = ffr.compute_probabilities(
    vix_state, np.percentile(vix_state, [25, 90]), half_life=T_tilde / 2)

S = 2500
H = 21

np.random.seed(0)  # for reproducibility
log_returns_sim = ffr.simulate(S, H, probs, states)
print(f'Simulated log returns dimension is {log_returns_sim.shape}')

Historical log returns dimension is (6812, 10)
Simulated log returns dimension is (2500, 10, 21)


In [5]:
cum_pnl = np.exp(np.cumsum(log_returns_sim, axis=2)) - 1
sim_stats_1m = ft.simulation_moments(
    pd.DataFrame(100 * cum_pnl[:, :, -1], columns=data.columns[:-1]))
np.round(sim_stats_1m, 2)

,Mean,Volatility,Skewness,Kurtosis
Ticker,,,,
Materials,1.63,5.79,-0.03,3.72
Energy,1.51,7.54,0.12,4.00
Financial,1.86,6.67,-0.04,4.96
Industrial,1.80,5.28,-0.25,4.23
Technology,2.16,6.68,-0.15,3.77
Consumer Staples,1.24,3.81,-0.13,3.71
Utilities,1.35,4.72,-0.00,3.99
Health Care,1.44,4.43,-0.20,3.73
Consumer Discretionary,1.91,5.86,-0.21,4.07


In [6]:
def portfolio_cml(
        cum_pnl: np.ndarray, portfolio: np.ndarray,
        alpha: float, p: np.ndarray) -> float:
    """Function for calculating the portfolio alpha-CML.

    Args:
        cum_pnl: Array of shape (S, I, H) with the cumulative pnl.
        portfolio: The portfolio of shape (I,).
        alpha: The alpha value.
        p: Probability vector of shape (S, 1).

    Returns:
        The alpha-CML for the portfolio.
    """
    results = np.full((S, H), np.nan)
    max_losses = np.full(S, np.nan)
    for s in range(S):
        results[s, :] = np.squeeze(cum_pnl[s, :, :].T @ portfolio)
        max_losses[s] = max(np.max(-cum_pnl[s, :, :].T @ portfolio), 0)
    worst_losses_inds = np.flip(np.argsort(max_losses))  # Worst losses first
    probs_sorted_cumsum = np.cumsum(p[worst_losses_inds])
    max_losses_sorted = max_losses[worst_losses_inds]
    alpha_ml_index = np.searchsorted(probs_sorted_cumsum, 1 - alpha, 'right')
    probs_total = probs_sorted_cumsum[alpha_ml_index - 1]
    if 1 - alpha - probs_total <= 1e-8:
        cml = (max_losses_sorted[:alpha_ml_index] @ p[worst_losses_inds][:alpha_ml_index]
               / probs_total)
    else:
        cml = ((max_losses_sorted[:alpha_ml_index] @ p[worst_losses_inds][:alpha_ml_index]
                + (1 - alpha - probs_total) * max_losses_sorted[alpha_ml_index])
               / (1 - alpha))
    return cml[0]

# CML optimization

In [7]:
def cml_matrices(
        p: np.ndarray, alpha: float, upper_bound: float, return_target: float = None
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """Function for building the matrices for the long-only Conditional Maximum Loss (CML)
    portfolio optimization using SciPy's linprog solver.

    Args:
        p: Array of shape (S, 1) with the path probailities.
        alpha: The alpha level for the CML optimization.
        upper_bound: The upper bound for the exposures.
        return_target: The return target. If None, the minimum CML portfolio
            is calculated. Default: None.

    Returns:
        A, b, G, h, and c where A and b are the input for the equality constraints,
        G and h are input for the inequality constraints, and c is the objective function.
    """
    num_vars = S + I + 1
    num_ineqs = S * H + I  # Lower bound of 0 is default in SciPy's linprog
    cum_losses = -cum_pnl

    # Objective function vector
    c = np.full((num_vars, 1), 0.)
    c[I:-1] = p / (1 - alpha)
    c[-1] = 1

    # Exposures must sum to 1
    A = np.zeros((1, num_vars))
    A[0, :I] = 1.
    b = np.array([[1.]])

    # Max loss exceeding the cumulative losses - MLaR for each simulated scenario
    G = np.full((num_ineqs, num_vars), 0.)
    for s in range(S):
        G[s * H:(s + 1) * H, :I] = cum_losses[s, :, :].T
        G[s * H:(s + 1) * H, I + s] = -np.ones(H)
    G[:S * H, -1] = -np.ones(S * H)
    h = np.full((num_ineqs, 1), 0.)
    
    # Upper bounds for the exposures
    G[-I:, :I] = np.identity(I)
    h[-I:, 0] = upper_bound

    # Optional return target
    if return_target is not None:
        expected_returns = p.T @ cum_pnl[:, :, -1]
        expected_return_row = np.zeros((1, num_vars))
        expected_return_row[0, :I] = expected_returns
        G = np.vstack((G, -expected_return_row))
        h = np.vstack((h, np.array([[-return_target]])))
    
    return A, b, G, h, c

In [8]:
p = np.full((S, 1), 1.) / S  # Uniform probabilities
individual_upper_bounds = 0.25
expected_returns = p.T @ cum_pnl[:, :, -1]
return_target = np.mean(expected_returns)
print(f'The portfolio return target is {return_target * 100:.4f}%')

The portfolio return target is 1.6486%


In [9]:
alpha = 0.9
start = time()
A, b, G, h, c = cml_matrices(
    p, alpha, individual_upper_bounds, return_target)
solution_full_cml_target = linprog(c, G, h, A, b)['x']
stop = time()
computation_time = round(stop - start, 2)
print(f'CML portfolio with a return target computed in {computation_time} seconds.')
solution_cml_target = solution_full_cml_target[:I]

CML portfolio with a return target computed in 3.74 seconds.


In [10]:
tol = 1e-10
assert return_target - (expected_returns @ solution_cml_target)[0] <= tol  # return target
assert np.abs(np.sum(solution_cml_target) - 1) <= tol  # self-financing
assert np.all(solution_cml_target >= 0 - tol)  # long-only

# CVaR optimization
For comparison, we compute the 90%-CVaR optimized portfolio for the last timestep, using the same bounds and return target.

Note that CVaR optimization is implemented using a faster algorithm, i.e., the one used for high-dimensional CVaR optimization in example 13: https://github.com/fortitudo-tech/fortitudo.tech/blob/main/examples/13_HighDimensionalCVaR.ipynb

In [11]:
# Set upper and lower bounds (same as for CML)
G_cvar = np.vstack((-np.identity(I), np.identity(I)))
h_cvar = np.hstack((np.zeros(I), np.full(I, individual_upper_bounds)))
opt = ft.MeanCVaR(cum_pnl[:, :, -1], G_cvar, h_cvar, alpha=alpha, options={'demean': False})

In [12]:
start = time()
solution_cvar_target = opt.efficient_portfolio(return_target)  # Same return target as EML
stop = time()
computation_time = round(stop - start, 2)
print(f'CVaR portfolio with a return target computed in {computation_time} seconds.')

CVaR portfolio with a return target computed in 0.01 seconds.


In [13]:
assert return_target - (expected_returns @ solution_cvar_target)[0] <= tol  # return target
assert np.abs(np.sum(solution_cvar_target) - 1) <= tol  # self-financing
assert np.all(solution_cvar_target >= 0 - tol)  # long-only

# Optimization with both CVaR and CML constraints

In [14]:
def cml_cvar_matrices(
        p: np.ndarray, alpha: float, upper_bound: float, cml_target: float, cvar_target: float
        ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    num_vars = 2 * S + I + 2
    num_ineqs = S * H + S + I + 2  # Lower bound of 0 is default in SciPy's linprog
    cum_losses = -cum_pnl

    # Objective function vector
    expected_returns = p.T @ cum_pnl[:, :, -1]
    c = np.full((num_vars, 1), 0.)
    c[:I, 0] = -expected_returns

    # Exposures must sum to 1
    A = np.zeros((1, num_vars))
    A[0, :I] = 1.
    b = np.array([[1.]])

    # CML constraint: Max loss exceeding the cumulative losses - alpha-ML
    G = np.full((num_ineqs, num_vars), 0.)
    for s in range(S):
        G[s * H:(s + 1) * H, :I] = cum_losses[s, :, :].T
        G[s * H:(s + 1) * H, I + s] = -np.ones(H)
    G[:S * H, -2] = -np.ones(S * H)
    h = np.full((num_ineqs, 1), 0.)

    # CML constraint: The CML target
    G[S * H, -2] = 1
    G[S * H, I:I + S] = np.squeeze(p / (1 - alpha))
    h[S * H, 0] = cml_target

    # CVaR constraint: Max loss exceeding the cumulative losses - VaR
    G[S * H + 1:S * H + 1 + S, :I] = cum_losses[:, :, -1]
    G[S * H + 1:S * H + 1 + S, I + S:I + 2 * S] = -np.identity(S)
    G[S * H + 1:S * H + 1 + S, -1] = -np.ones(S)

    # CVaR constraint: The CVaR target
    G[S * H + 1 + S, -1] = 1
    G[S * H + 1 + S, I + S:I + 2 * S] = np.squeeze(p / (1 - alpha))
    h[S * H + 1 + S, 0] = cvar_target

    # Upper bounds for the exposures
    G[-I:, :I] = np.identity(I)
    h[-I:, 0] = upper_bound

    return A, b, G, h, c

In [15]:
cml_target = portfolio_cml(cum_pnl, solution_cml_target, alpha, p)
print(f'CML target is {np.round(100 * cml_target, 4)}%')

CML target is 7.4881%


In [16]:
cvar_target = ft.portfolio_cvar(
    solution_cvar_target, cum_pnl[:, :, -1], alpha=alpha, demean=False)
print(f'CVaR target is {np.round(100 * cvar_target, 4)}%')

CVaR target is 6.2401%


In [17]:
# Make VaR unbounded (SciPy solver has VaR >= 0 by default)
bounds = [(0, None)] * (2 * S + I + 1)
bounds.append((None, None))

In [18]:
start = time()
A, b, G, h, c = cml_cvar_matrices(
    p, alpha, individual_upper_bounds, cml_target, cvar_target)  # Uniform probs and same bounds as before
solution_full_double_target = linprog(c, G, h, A, b, bounds)['x']
stop = time()
computation_time = round(stop - start, 2)
print(f'Max return portfolio with a CVaR and CML target computed in {computation_time} seconds.')
solution_double_target = solution_full_double_target[:I]

Max return portfolio with a CVaR and CML target computed in 4.76 seconds.


In [19]:
double_target_cml = portfolio_cml(cum_pnl, solution_double_target, alpha, p)
print(f'{alpha * 100}%-CML for CVaR and CML optimized portfolio: {double_target_cml}. We expect it to be at most {cml_target}.')

solution_double_target = solution_double_target[:, np.newaxis]
double_target_cvar = ft.portfolio_cvar(
    solution_double_target, cum_pnl[:, :, -1], alpha=alpha, demean=False)
print(f'{alpha * 100}%-CVaR for CVaR and CML optimized portfolio: {double_target_cvar}. We expect it to be at most {cvar_target}.')

90.0%-CML for CVaR and CML optimized portfolio: 0.07488067912841385. We expect it to be at most 0.0748806791284134.
90.0%-CVaR for CVaR and CML optimized portfolio: 0.06240058502986839. We expect it to be at most 0.06240058502986823.


# Compare portfolios

In [20]:
solutions = np.hstack(
    (solution_cml_target[:, np.newaxis], solution_cvar_target, solution_double_target))
expected_returns_pfs = expected_returns @ solutions
cvars = ft.portfolio_cvar(solutions, cum_pnl[:, :, -1], p, alpha, demean=False)
cmls = np.array(
    [portfolio_cml(cum_pnl, solutions[:, i], alpha, p) for i in range(3)])

pd.DataFrame(np.round(100 * np.vstack((solutions, expected_returns_pfs, cvars, cmls)), 4),
            index=list(data.columns[0:-1]) + ['Expected return', 'CVaR', 'CML'],
            columns=['CML', 'CVaR', 'CML + CVaR'])

,CML,CVaR,CML + CVaR
Materials,0.0000,0.0000,0.0000
Energy,0.0000,0.0000,0.0000
Financial,0.0000,0.0000,0.0000
Industrial,10.8657,8.3453,8.6012
Technology,25.0000,24.3617,25.0000
Consumer Staples,21.3601,14.1271,17.6321
Utilities,21.0314,25.0000,22.7975
Health Care,10.8080,16.6570,14.7578
Consumer Discretionary,10.9347,11.5090,11.2114
S&P 500,0.0000,0.0000,0.0000


# Sequential Entropy Pooling (SeqEP) views
To illustrate how to optimize the CML with non-uniform joint scenario probabilities, we use Sequential Entropy Pooling (SeqEP) to implement some market views.

For more on SeqEP, see Chapter 5 of the Portfolio Construction and Risk Management book: https://antonvorobets.substack.com/p/pcrm-book

You can also read the original article here: https://ssrn.com/abstract=3936392

The example below essentially uses the code from https://github.com/fortitudo-tech/fortitudo.tech/blob/main/examples/2_SequentialEntropyPooling.ipynb

In [21]:
R = cum_pnl[:, :, -1]

# C0 views
mean_row = R[:, -1][np.newaxis]  # S&P 500
A0 = np.vstack((np.ones((1, S)), mean_row))
snp_mean_view = sim_stats_1m['Mean'].values[-1] * 0.9 / 100
b0 = np.array([[1.], [snp_mean_view]])
q0 = ft.entropy_pooling(p, A0, b0)

In [22]:
# Verify that the mean view is satisfied
stats0 = ft.simulation_moments(pd.DataFrame(R, columns=data.columns[:-1]), q0)
print(f'Difference between mean view and C0 posterior mean value is {stats0['Mean'].values[-1] - snp_mean_view}')
means0 = stats0['Mean'].values

Difference between mean view and C0 posterior mean value is -1.7080796846369317e-11


In [23]:
means0 = q0.T @ R
vol_row0 = (R[:, 4] - means0[0, 4])[np.newaxis]**2  # Technology
mean_rows = R[:, [4, 9]].T
A1 = np.vstack((np.ones((1, S)), mean_rows, vol_row0))
tech_variance_view = (1.1 * sim_stats_1m['Volatility'].values[4] / 100)**2
b1 = np.array([[1.], [means0[0, 4]], [means0[0, 9]], [tech_variance_view]])
q1 = ft.entropy_pooling(p, A1, b1)

In [24]:
# Verify that the mean and volatility views are satisfied
stats1 = ft.simulation_moments(pd.DataFrame(R, columns=data.columns[:-1]), q1)
print(f'Difference between volatility view and posterior is {stats1['Volatility'].values[4] - np.sqrt(tech_variance_view)}')

Difference between volatility view and posterior is 2.6150664966806403e-11


In [25]:
# Overview of posterior simulation moments
ft.simulation_moments(pd.DataFrame(100 * R, columns=data.columns[:-1]), q1)

,Mean,Volatility,Skewness,Kurtosis
Ticker,,,,
Materials,1.487620,6.062108,-0.087847,3.724812
Energy,1.360084,7.794872,0.083387,4.080262
Financial,1.711992,7.054340,-0.071483,5.052463
Industrial,1.650329,5.638565,-0.300396,4.296310
Technology,1.963589,7.350176,-0.192906,3.862521
Consumer Staples,1.161738,3.915579,-0.174804,3.724841
Utilities,1.257642,4.851919,-0.059027,4.078745
Health Care,1.336885,4.630341,-0.263963,3.782539
Consumer Discretionary,1.732535,6.261434,-0.315324,4.212407


In [26]:
# Assess the significance of the view with effective number of scenarios
relative_entropy = q1.T @ (np.log(q1) - np.log(p))
effective_number_scenarios = np.exp(-relative_entropy)
print(f'Effective number of scenarios (ENS) is {np.round(100 * effective_number_scenarios[0, 0], 2)}%')

Effective number of scenarios (ENS) is 99.29%


# Posterior optimization
With the new scenario probabilities at hand, we once again perform portfolio optimization of CML, CVaR and CML + CVaR.

In [27]:
expected_returns_post = stats1['Mean'].values
return_target_post = np.mean(expected_returns_post)
print(f'Return target for posterior optimization is {np.round(100 * return_target_post, 4)}%.')

Return target for posterior optimization is 1.5086%.


In [28]:
start = time()
A, b, G, h, c = cml_matrices(
    q1, alpha, individual_upper_bounds, return_target_post)
solution_full_cml_target_post = linprog(c, G, h, A, b)['x']
stop = time()
computation_time = round(stop - start, 2)
print(f'CML portfolio with a return target computed in {computation_time} seconds.')
solution_cml_target_post = solution_full_cml_target_post[:I]

CML portfolio with a return target computed in 3.39 seconds.


In [29]:
assert return_target_post - (expected_returns_post @ solution_cml_target_post) <= tol  # return target
assert np.abs(np.sum(solution_cml_target_post) - 1) <= tol  # self-financing
assert np.all(solution_cml_target_post >= 0 - tol)  # long-only

In [30]:
opt_post = ft.MeanCVaR(
    cum_pnl[:, :, -1], G_cvar, h_cvar, p=q1,
    alpha=alpha, options={'demean': False})

In [31]:
start = time()
solution_cvar_target_post = opt_post.efficient_portfolio(return_target_post)  # Same return target as CML
stop = time()
computation_time = round(stop - start, 2)
print(f'CVaR portfolio with a return target computed in {computation_time} seconds.')

CVaR portfolio with a return target computed in 0.01 seconds.


In [32]:
assert return_target_post - (expected_returns_post @ solution_cvar_target_post)[0] <= tol  # return target
assert np.abs(np.sum(solution_cvar_target_post) - 1) <= tol  # self-financing
assert np.all(solution_cvar_target_post >= 0 - tol)  # long-only

In [33]:
cml_target_post = portfolio_cml(cum_pnl, solution_cml_target_post, alpha, q1)
print(f'CML target is {np.round(100 * cml_target_post, 4)}%')

CML target is 8.2489%


In [34]:
cvar_target_post = ft.portfolio_cvar(
    solution_cvar_target_post, cum_pnl[:, :, -1], p=q1, alpha=alpha, demean=False)
print(f'CML target is {np.round(100 * cvar_target_post, 4)}%')

CML target is 7.1333%


In [35]:
start = time()
A, b, G, h, c = cml_cvar_matrices(
    q1, alpha, individual_upper_bounds, cml_target_post, cvar_target_post)
solution_full_double_target_post = linprog(c, G, h, A, b, bounds)['x']
stop = time()
computation_time = round(stop - start, 2)
print(f'Max return portfolio with a CVaR and CML target computed in {computation_time} seconds.')
solution_double_target_post = solution_full_double_target_post[:I]

Max return portfolio with a CVaR and CML target computed in 3.96 seconds.


In [36]:
solutions_post = np.hstack(
    (solution_cml_target_post[:, np.newaxis], solution_cvar_target_post, solution_double_target_post[:, np.newaxis]))
expected_returns_pfs_post = expected_returns_post @ solutions_post
cvars_post = ft.portfolio_cvar(solutions_post, cum_pnl[:, :, -1], q1, alpha, demean=False)
cmls_post = np.array(
    [portfolio_cml(cum_pnl, solutions_post[:, i], alpha, q1) for i in range(3)])

pd.DataFrame(
    np.round(100 * np.vstack((solutions_post, expected_returns_pfs_post, cvars_post, cmls_post)), 4),
    index=list(data.columns[0:-1]) + ['Expected return', 'CVaR', 'CML'],
    columns=['CML', 'CVaR', 'CML + CVaR'])

,CML,CVaR,CML + CVaR
Materials,0.0000,0.0000,0.0000
Energy,0.0000,0.0000,0.0000
Financial,0.0000,0.0000,0.0000
Industrial,13.7428,8.4690,11.1074
Technology,25.0000,24.8291,25.0000
Consumer Staples,22.1332,16.4720,19.9443
Utilities,22.5990,25.0000,24.5376
Health Care,9.2751,15.5564,10.7339
Consumer Discretionary,7.2499,9.6735,8.6768
S&P 500,0.0000,0.0000,0.0000
